<a href="https://colab.research.google.com/github/muhamadrefo/ImageClassificationColab/blob/master/CSV_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Environment 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
#Importing Packages

import numpy as np
import matplotlib.pyplot as plt
import time
import glob
import math
import cv2
import os
import pandas as pd

from PIL import Image
from PIL import ImageDraw
from datetime import datetime
from skimage.transform import resize
from skimage.feature import greycomatrix, greycoprops
#from sklearn.decomposition import PCA

In [3]:
map_8bit_to_3bit = [i // 32 for i in range(256)]

# Define Function Preprocessing

In [4]:
def read_file(myDir):
  file_name = []
  label = []
  namePath = []

  for path, subdirs, files in os.walk(myDir):
    for name in files:
      namePath.append(os.path.join(path, name))
      label.append(path.split('/')[-1])

  return namePath, label


def load_img(img_path):
  img = Image.open(img_path).convert('L')
  resultImg = img.resize((224, 224))
  return resultImg


def show_img(img):
  plt.imshow(img, cmap='gray')

def getLabels(path):
  label = os.listdir(path)
  return label


In [5]:
#Preprocessing start from here

def getColoringIMG(img):
  ColoringIMG = list(img.getdata())
  return ColoringIMG


def mapTo3bit(ColoringIMG):
  i = 0

  for colorIMG in ColoringIMG:
    ColoringIMG[i] = map_8bit_to_3bit[colorIMG]
    i += 1

  return ColoringIMG


def transToMatriks(ColoringIMG):
  matriksIMG = []
  i = 0

  dimensionIMG = int(math.sqrt(len(ColoringIMG)))

  for row in range(dimensionIMG):
    newRow = []
    for col in range(dimensionIMG):
      newRow.append(ColoringIMG[i])
      i += 1
    matriksIMG.append(newRow)

  return matriksIMG

def ExtractIMG(img):
  IMGColor = getColoringIMG(img)
  _3BitColor = mapTo3bit(IMGColor)
  matriksIMG = transToMatriks(_3BitColor)

  GLCMresult = []
  glcm = greycomatrix(matriksIMG, distances=[1],angles=[0],levels=256,symmetric=False,normed=False)
  komponen = ['dissimilarity', 'contrast', 'homogeneity', 'energy', 'correlation', 'ASM']

  for i in komponen:
    GLCMresult.append(greycoprops(glcm, i)[0][0])

  return GLCMresult

# Train here to get dataset.csv

In [6]:
%cd /content/drive/My Drive/Batik/

/content/drive/My Drive/Batik


In [7]:
#path_train = '/content/drive/My Drive/TestingPlace/PlaceofDatabase/train'
'''
Using 'CNN/DataTrain/' on dataroot will give you result as dataset train
Using 'CNN/DataValid/' on dataroot will give you result as dataset validation
'''
datasetRoot = 'CNN/DataValid/'

In [8]:
startTimeModule = datetime.now()

path_trainIMG = []
label_trainIMG = []
dissimilarity = []
contrast = []
homogeneity = []
energy = []
correlation = []
ASM = []

labels = getLabels(datasetRoot)

for label in labels:
  names_Path = glob.glob(datasetRoot + label + '/' + '*.jpg')
  for namePath in names_Path:
    path_trainIMG.append(namePath)
    label_trainIMG.append(label)
    trainImg = load_img(namePath)
    featureOfImg = ExtractIMG(trainImg)
    dissimilarity.append(featureOfImg[0])
    contrast.append(featureOfImg[1])
    homogeneity.append(featureOfImg[2])
    energy.append(featureOfImg[3])
    correlation.append(featureOfImg[4])
    ASM.append(featureOfImg[5])

columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
#['Label','Path', 'Homogeneity', 'Dissimilarity', 'Contrast', 
#           'Energy', 'Correlation', 'ASM']
variables = {
    'Path':path_trainIMG,
    'Contrast':contrast,
    'Dissimilarity':dissimilarity,
    'Homogeneity':homogeneity,
    'Energy':energy,
    'Correlation':correlation,
    'ASM':ASM,
    'Label':label_trainIMG,
             }

data_train = pd.DataFrame(variables, columns = columns)
data_train.to_csv("DataValidation.csv", index=False) #Changes DataTraining.csv to DataValidation.csv or vice versa
print('Total time for processing :', (datetime.now() - startTimeModule))

Total time for processing : 0:01:02.105926
